In [1]:
# Import Statements
from sqlalchemy import create_engine, MetaData, Table
import json
import pandas as pd

In [2]:
# Database Variables
db_url = "..."
table_name = "JOR"
data_column_name = 'datastring'

In [3]:
# boilerplace sqlalchemy setup
engine = create_engine(db_url)
metadata = MetaData()
metadata.bind = engine
table = Table(table_name, metadata, autoload=True)

In [4]:
# make a query and loop through
s = table.select()
rows = s.execute()
print(type(rows))
data = []
statuses = [0,1, 2,3,4,5,6,7] #status codes of subjects who completed experiment
exclude = [] # if you have workers you wish to exclude, add them here
conditions = []

<class 'sqlalchemy.engine.cursor.LegacyCursorResult'>


In [5]:
# count = 0
# for row in rows:
#     if row['mode'] != 'live':
#         print(row['mode'])
        
#     if row['status'] == 3:
#         print(row['status'])
#     if count > 600:
#         if row['status'] == 3:
#             print(row['datastring']['id'])
        
#     if count == 900:
#         print(row['codeversion'])
#         for i in dict(row):
#             print(i)
#     count += 1

In [6]:
for row in rows:
    # only use subjects who completed experiment and aren't excluded
    if row['status'] in statuses and row['uniqueid'] not in exclude:
        conditions.append({
                'uniqueid': row['uniqueid'],
                'beginhit': row['beginhit'],
                'endhit': row['endhit'],
                'status': row['status'],
                'counterbalance': row['counterbalance'],
                'mode': row['mode'],
                'workerid': row['workerid'],
                'codeversion': row['codeversion']
            })
        
        
        data.append(row[data_column_name])

print(len(data))

1319


In [11]:
condition_frame = pd.DataFrame(conditions)
condition_frame[condition_frame['codeversion'] == '2.0']

,uniqueid,beginhit,endhit,status,counterbalance,mode,workerid,codeversion
40,A13PQTFAQ5LFSZ:3W8CV64QJ36XNYFL72PIVYKJO9YH9W,2021-08-03 23:50:20,NaT,6,2,live,A13PQTFAQ5LFSZ,2.0
316,A1XTT903BSWHYZ:38BQUHLA9X7MMZK20OODKTN40UJOMQ,2021-08-03 23:00:52,2021-08-03 23:22:41,4,3,live,A1XTT903BSWHYZ,2.0
610,A2SLA2D0C7OGB9:3W8CV64QJ36XNYFL72PIVYKJO9YH9W,2021-08-04 00:46:16,NaT,6,1,live,A2SLA2D0C7OGB9,2.0
673,A2XISYL8BR08AV:38BQUHLA9X7MMZK20OODKTN40UJOMQ,2021-08-03 20:59:11,NaT,6,1,live,A2XISYL8BR08AV,2.0
710,A313TT0C44PT1Y:3VNL7UK1XGQWTHXTAI6ZJD9I21OTFA,2021-08-03 23:27:22,2021-08-03 23:44:34,4,0,live,A313TT0C44PT1Y,2.0
832,A3DE5N73C0JHU9:3VNL7UK1XGQWTHXTAI6ZJD9I21OTFA,2021-08-03 20:43:59,NaT,6,0,live,A3DE5N73C0JHU9,2.0


In [8]:
checks = ['A3MYPYBVHX7FQ2', 'A2YWLCSIANMV1S', 'A7FQJ15Y5XV6W', 'A1SLD1XTQAUGH', 'A2ZHBK2MDL2L51', ]
for i in checks:
    print(i)
    display(condition_frame[condition_frame['workerid'] == i])

A3MYPYBVHX7FQ2


,uniqueid,beginhit,endhit,status,counterbalance,mode,workerid,codeversion
919,A3MYPYBVHX7FQ2:3DPNQGW4LMMGT3PSUO2NHZKDNH464A,2021-06-29 16:05:07,NaT,2,44,live,A3MYPYBVHX7FQ2,1.0


A2YWLCSIANMV1S


,uniqueid,beginhit,endhit,status,counterbalance,mode,workerid,codeversion
693,A2YWLCSIANMV1S:35GCEFQ6I6VXZHQZWFUF4SL2ELIZ3M,2021-06-29 14:24:59,NaT,6,7,live,A2YWLCSIANMV1S,1.0


A7FQJ15Y5XV6W


,uniqueid,beginhit,endhit,status,counterbalance,mode,workerid,codeversion
1036,A7FQJ15Y5XV6W:33CUSNVVNOJ4GIVIZ6PYIPW9VY488J,2021-06-29 13:30:10,2021-06-29 15:23:04,5,36,live,A7FQJ15Y5XV6W,1.0


A1SLD1XTQAUGH


,uniqueid,beginhit,endhit,status,counterbalance,mode,workerid,codeversion
274,A1SLD1XTQAUGH:3HMVI3QICKZ1KO2FDPED2SX3A711Y6,2021-06-29 13:30:20,2021-06-29 15:00:59,4,72,live,A1SLD1XTQAUGH,1.0


A2ZHBK2MDL2L51


,uniqueid,beginhit,endhit,status,counterbalance,mode,workerid,codeversion
699,A2ZHBK2MDL2L51:3HL8HNGX468D92OFCC1ISD8609PF9M,2021-06-10 17:02:03,2021-06-10 17:29:21,5,72,live,A2ZHBK2MDL2L51,1.0


In [9]:
subject_data = []
for subject_json in data:
    try:
        subject_dict = json.loads(subject_json)
        subject_data.append(subject_dict['data'])
    except:
        continue

In [10]:
for part in subject_data:
    for record in part:
        record['trialdata']['uniqueid'] = record['uniqueid']